## Work Notebook
### Algorithm of Hybrid Rec Engine - find similar products according a tag classifier followed by cosine similarity by Spacy embeddings of reviewText 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [167]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

  0%|                          | 200/1689188 [12:23:59<104716:03:48, 223.20s/it]


### Loading pickled webscraping files

In [2]:
with open('load_df.pickle', 'rb') as file:
    load_df = pickle.load(file)

### We will train a model that classifies user input (item) into appropriate tag labels 

In [3]:
load_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,item,tag,price
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011",[Rand McNally 528881469 7-inch Intelliroute TN...,Trucking GPS,$399.99


In [4]:
item_list=[]

for i in range(702972):
    try:
        if len(load_df.item[i]) >= 1:
            item_list.append(load_df.item[i][0])
    except:
        item_list.append(np.nan)
        continue
        


In [5]:
load_df['item']=pd.Series(item_list)

In [6]:
load_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,item,tag,price
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013",Rand McNally 528881469 7-inch Intelliroute TND...,Trucking GPS,$399.99
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010",Rand McNally 528881469 7-inch Intelliroute TND...,Trucking GPS,$399.99
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010",Rand McNally 528881469 7-inch Intelliroute TND...,Trucking GPS,$399.99
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010",Rand McNally 528881469 7-inch Intelliroute TND...,Trucking GPS,$399.99
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011",Rand McNally 528881469 7-inch Intelliroute TND...,Trucking GPS,$399.99


In [7]:
df_mod=load_df[['item','tag']]

In [8]:
df_mod.dropna(inplace=True)

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
df_mod.drop_duplicates(subset='item')
df_mod.shape

(690574, 2)

In [10]:
y=df_mod.tag

In [11]:
X=df_mod.item

In [12]:
import re
X = [re.sub(r"\d+", '', x) for x in X]

In [13]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=0)

In [15]:
#using tf_idf vectorizer
stop_words = set(stopwords.words("english"))
tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,2), lowercase=True)
tfidf_data_train_lem = tfidf.fit_transform(X_train)
tfidf_data_test_lem = tfidf.transform(X_test)


### Multinomial Naive Bayes

In [26]:
from sklearn.naive_bayes import MultinomialNB  
classifier = MultinomialNB(alpha=0)  
classifier.fit(tfidf_data_train_lem, y_train)  

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


MultinomialNB(alpha=0)

In [27]:
# y_preds=classifier.predict(tfidf_data_test_lem)
y_pred=classifier.predict(tfidf_data_test_lem)

In [28]:
y_pred

array(['Mice', 'External Solid State Drives', 'Network Adapters', ...,
       'CD & DVD Drives', 'SD Cards', 'Radio Scanners'], dtype='<U46')

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics

In [30]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.f1_score(y_test, y_pred, average='weighted'))


0.7046540589070057
0.7355476787265816


In [21]:
classifier

MultinomialNB(alpha=0)

### Multinomial NB with Gridsearch

In [31]:
from sklearn.model_selection import GridSearchCV
NB_params = [{'alpha': [.0001, .001, .01, 0, 1, 10, 15, 5]}]

NB_clf = GridSearchCV(classifier, NB_params, cv = 3, scoring='accuracy')

NB_clf.fit(tfidf_data_train_lem, y_train)

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


GridSearchCV(cv=3, estimator=MultinomialNB(alpha=0),
             param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0, 1, 10, 15]}],
             scoring='accuracy')

In [32]:
nb_preds=NB_clf.predict(tfidf_data_test_lem)

In [33]:
#Evaluation metrics
print(metrics.accuracy_score(y_test, nb_preds))
print(metrics.f1_score(y_test, nb_preds, average='weighted'))

0.7145298155173911
0.7207195947389221


### SVM

In [36]:
#starting SVM model
from sklearn import svm

In [37]:
#setting hyperparameters for the SVM model
svm_clf = svm.SVC(kernel='rbf', class_weight='balanced')

In [ ]:
#fitting the SVM model
svm_clf.fit(tfidf_data_train_lem, y_train)

In [ ]:
#predicting the classification with SVM
svm_preds=svm_clf.predict(tfidf_data_test_lem)

In [ ]:
#Evaluation metrics
print(metrics.accuracy_score(y_test, svm_preds))
print(metrics.f1_score(y_test, svm_preds, average='weighted'))

### Random Forest

In [17]:
tfidf_data_train_lem

<656045x100173 sparse matrix of type '<class 'numpy.float64'>'
	with 12022642 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.ensemble import RandomForestClassifier
#initializing RandomFoarest 
rfc = RandomForestClassifier(n_estimators=200, criterion='entropy',random_state=0, n_jobs=-1, verbose=2)

In [20]:
#fitting model
rfc.fit(tfidf_data_train_lem, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 300building tree 2 of 300

building tree 3 of 300building tree 4 of 300building tree 5 of 300
building tree 6 of 300


building tree 7 of 300building tree 8 of 300building tree 9 of 300

building tree 10 of 300

building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300


[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 13.9min


building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of 300
building tree 49 of 300
building tree 50 of 300
building tree 51 of 300
building tree 52 of 300
building tree 53 of 300
building tree 54 of 300
building tree 55 of 300
building tree 56 of 300
building tree 57 of 300
building tree 58 of 300
building tree 59 of 300
building tree 60 of 300
building tree 61 of 300
building tree 62 of 300
building tree 63 of 300
building tree 64 of 300
building tree 65 of 300
building tree 66 of 300
building tree 67 of 300
building tree 68 of 300
building tree 69 of 300
building tree 70 of 300
building tree 71 of 300
building tree 72 of 300
building tree 73

[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed: 70.2min


building tree 153 of 300
building tree 154 of 300
building tree 155 of 300
building tree 156 of 300
building tree 157 of 300
building tree 158 of 300
building tree 159 of 300
building tree 160 of 300
building tree 161 of 300
building tree 162 of 300
building tree 163 of 300
building tree 164 of 300
building tree 165 of 300
building tree 166 of 300
building tree 167 of 300
building tree 168 of 300
building tree 169 of 300
building tree 170 of 300
building tree 171 of 300
building tree 172 of 300
building tree 173 of 300
building tree 174 of 300
building tree 175 of 300
building tree 176 of 300
building tree 177 of 300
building tree 178 of 300
building tree 179 of 300
building tree 180 of 300
building tree 181 of 300
building tree 182 of 300
building tree 183 of 300
building tree 184 of 300
building tree 185 of 300
building tree 186 of 300
building tree 187 of 300
building tree 188 of 300
building tree 189 of 300
building tree 190 of 300
building tree 191 of 300
building tree 192 of 300


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 141.8min finished


RandomForestClassifier(criterion='entropy', n_estimators=300, n_jobs=-1,
                       random_state=0, verbose=2)

In [23]:
rfc_preds=rfc.predict(tfidf_data_test_lem)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    2.0s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:    9.5s
[Parallel(n_jobs=10)]: Done 300 out of 300 | elapsed:   19.6s finished


In [26]:
#Evaluation metrics
print(metrics.accuracy_score(y_test, rfc_preds))
print(metrics.f1_score(y_test, rfc_preds, average='weighted'))

0.7796055489588462
0.7771464903277842


### Random Forest w/ Gridsearch CV

In [16]:
rfc_gs= RandomForestClassifier(min_samples_split=2, random_state=0, n_jobs=-1, verbose=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
forest_params = [{'warm_start': [True, False], 'n_estimators': [300, 350, 400, 500], 'criterion': ['gini', 'entropy'] }]

clf = GridSearchCV(rfc_gs, forest_params, cv = 3, scoring='accuracy')

clf.fit(tfidf_data_train_lem, y_train)

/Users/jeffreyng/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed: 46.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 77.1min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   36.2s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  2.8min
[Parallel(n_jobs=10)]: Done 300 out of 300 | elapsed:  4.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


In [34]:
gs_preds=clf.predict(tfidf_data_test_lem)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    4.1s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:    5.0s finished


In [35]:
#Evaluation metrics
print(metrics.accuracy_score(y_test, gs_preds))
print(metrics.f1_score(y_test, gs_preds, average='weighted'))

0.14292333980132643
0.13081426532121887


### We will find the sentiment score for each review 

In [37]:
df.shape

(63001, 9)

In [53]:
df.reviewText

0          0    We got this GPS for my husband who is an ...
5          5    I am using this with a Nook HD+. It works...
10         10    This item is just as was described in th...
18         18     This is a great buy, compared to a $60 ...
237        237    The Nook Tablet, in both the 16gb versi...
                                 ...                        
1689106    1689106    The Sabrent CR-CCU3 Card reader is ...
1689125    1689125    Great hub for carrying around if yo...
1689142    1689142    This is an interesting device in th...
1689162    1689162    ...all part of the package that com...
1689183    1689183    Burned these in before listening to...
Name: reviewText, Length: 63001, dtype: object

In [71]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [95]:
sent=[sid.polarity_scores(x) for x in tqdm(df.reviewText)]

100%|████████████████████████████████| 1689188/1689188 [31:06<00:00, 904.93it/s]


In [110]:
df['vader_score']=[sent[x].get('compound') for x in range(len(sent))]

In [113]:
# Mapped Vader scores to integer counts for easier manipulation 
col         = 'vader_score'
conditions  = [ df[col] >= .15, (df[col] < .1) & (df[col]> -.1), df[col] <= -.1 ]
choices     = [ 1, 0, -1 ]
df["vader_score"] = np.select(conditions, choices, default=np.nan)  

In [115]:
#final mapping process
df['vader_score']= df['vader_score'].map({-1:'negative', 0: 'neutral', 1:'positive'})

In [117]:
df.vader_score.value_counts()

positive    1399723
negative     194525
neutral       80603
Name: vader_score, dtype: int64

### We will use Spacy Embeddings for the df.reviewText

In [118]:
#importing spacy 
import spacy
nlp = spacy.load("en_core_web_lg")

In [119]:
#tokenizer
spacy_tokenizer = nlp.tokenizer

In [120]:
#function to take reviewText column an return word embedding
def prep(x):
    z=spacy_tokenizer(x) 
    z=nlp(z).vector.reshape(300,)
    return z


### I also tried gensim for word embeddings

In [125]:
import gensim
# set the correct path to the file on your machine
model = gensim.models.KeyedVectors.load_word2vec_format('~/Downloads/crawl-300d-2M.vec', binary=False)

In [145]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.key_to_index]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [156]:
vec = [get_mean_vector(model, df.reviewText[i]) for i in tqdm(range(len(df.reviewText)))]
vec

In [ ]:
len(vec)

In [158]:
len(vec[0])

300

### The recommendation algo 

In [472]:
ser=pd.Series(['gps'])
dat=pd.DataFrame(ser)
dat.rename(columns={0:'inventory'}, inplace=True)

In [475]:
str(y_pred[0])

'Handheld GPS Units'

In [ ]:
#We enter our keyword search and create a df to be predicted upon
tfidf_data_lem = tfidf.transform(dat.inventory)
#predict our df
y_pred=rfc.predict(tfidf_data_lem)
#finds similar product content based 
new_df=load_df[load_df.tag==str(y_pred[0])]
#asks user to find the price range 
selected=new_df[new_df['price']<'$200.00'].reviewText
#asks user to select from a given set of items 


#word embeddings calculated for cosine similarity
rev_vec=[prep(x) for x in selected]


In [504]:
new_df.item

1269        [Magellan GPS 315 Waterproof Hiking GPS]
1270        [Magellan GPS 315 Waterproof Hiking GPS]
1271        [Magellan GPS 315 Waterproof Hiking GPS]
1272        [Magellan GPS 315 Waterproof Hiking GPS]
1273        [Magellan GPS 315 Waterproof Hiking GPS]
                             ...                    
700258    [Garmin GPSMAP 62S Handheld GPS Navigator]
700259    [Garmin GPSMAP 62S Handheld GPS Navigator]
700260    [Garmin GPSMAP 62S Handheld GPS Navigator]
700261    [Garmin GPSMAP 62S Handheld GPS Navigator]
700262    [Garmin GPSMAP 62S Handheld GPS Navigator]
Name: item, Length: 6058, dtype: object

In [509]:
new_df[new_df['item']=='Magellan GPS 315 Waterproof Hiking GPS']


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,item,tag,price


In [505]:
q= new_df[new_df['item']=='Magellan GPS 315 Waterproof Hiking GPS'].reviewText

ref_vec=prep(q[0])

listA=[]
for i in range(len(rev_vec)):
    listA.append(np.dot(rev_vec[i], ref_vec )/(np.linalg.norm(rev_vec[i])*np.linalg.norm(ref_vec)))
    ser= listA
    new_df['cosine_sim']=ser

KeyError: 0